# CS483 - Colab 3
## K-Means & PCA

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=58a61406d465f624206b24b7513d047f3620f955339dfa04bbd251777a31d87a
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 39.6 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

Now we import some of the libraries usually needed by our workload.





In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [3]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Data Preprocessing

In this Colab, rather than downloading a file from Google Drive, we will load a famous machine learning dataset, the [Breast Cancer Wisconsin dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html), using the ```scikit-learn``` datasets loader.

In [4]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()

For convenience, given that the dataset is small, we first

*   construct a Pandas dataframe
*   tune the schema
*   and convert it into a Spark dataframe.

In [5]:
pd_df = pd.DataFrame(breast_cancer.data, columns=breast_cancer.feature_names)
df = spark.createDataFrame(pd_df)

def set_df_columns_nullable(spark, df, column_list, nullable=False):
    for struct_field in df.schema:
        if struct_field.name in column_list:
            struct_field.nullable = nullable
    df_mod = spark.createDataFrame(df.rdd, df.schema)
    return df_mod

df = set_df_columns_nullable(spark, df, df.columns)
df = df.withColumn('features', array(df.columns))
vectors = df.rdd.map(lambda row: Vectors.dense(row.features))

df.printSchema()

root
 |-- mean radius: double (nullable = false)
 |-- mean texture: double (nullable = false)
 |-- mean perimeter: double (nullable = false)
 |-- mean area: double (nullable = false)
 |-- mean smoothness: double (nullable = false)
 |-- mean compactness: double (nullable = false)
 |-- mean concavity: double (nullable = false)
 |-- mean concave points: double (nullable = false)
 |-- mean symmetry: double (nullable = false)
 |-- mean fractal dimension: double (nullable = false)
 |-- radius error: double (nullable = false)
 |-- texture error: double (nullable = false)
 |-- perimeter error: double (nullable = false)
 |-- area error: double (nullable = false)
 |-- smoothness error: double (nullable = false)
 |-- compactness error: double (nullable = false)
 |-- concavity error: double (nullable = false)
 |-- concave points error: double (nullable = false)
 |-- symmetry error: double (nullable = false)
 |-- fractal dimension error: double (nullable = false)
 |-- worst radius: double (nullable

With the next cell, we build the two data structures that we will be using throughout this Colab:


*   ```features```, a dataframe of Dense vectors, containing all the original features in the dataset;
*   ```labels```, a series of binary labels indicating if the corresponding set of features belongs to a subject with breast cancer, or not.



In [6]:
from pyspark.ml.linalg import Vectors
features = spark.createDataFrame(vectors.map(Row), ["features"])
labels = pd.Series(breast_cancer.target)

### Your task

If you run successfully the Setup and Data Preprocessing stages, you are now ready to cluster the data with the [K-means](https://spark.apache.org/docs/latest/ml-clustering.html) algorithm included in MLlib (Spark's Machine Learning library).
Set the ```k``` parameter to **2**, fit the model, and the compute the [Silhouette score](https://en.wikipedia.org/wiki/Silhouette_(clustering)) (i.e., a measure of quality of the obtained clustering).  

**IMPORTANT:** use the MLlib implementation of the Silhouette score (via ```ClusteringEvaluator```).

In [30]:
''' 8-9 lines of code in total expected but can differ based on your style.
for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

kmeans=KMeans().setK(2).setSeed(1)
kmeans.getFeaturesCol()
model = kmeans.fit(features)
predictions = model.transform(features)
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette score = ", silhouette)
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Silhouette score =  0.8342904262826145
Cluster Centers: 
[1.25562991e+01 1.85703653e+01 8.11234703e+01 4.96061872e+02
 9.48844977e-02 9.10998174e-02 6.24377642e-02 3.34325434e-02
 1.78057991e-01 6.34540183e-02 3.04190868e-01 1.21515320e+00
 2.15288059e+00 2.37852922e+01 7.17326256e-03 2.34746895e-02
 2.87455128e-02 1.06363242e-02 2.06135799e-02 3.74750297e-03
 1.40439018e+01 2.47095434e+01 9.19375114e+01 6.19647945e+02
 1.29959110e-01 2.23311758e-01 2.19214947e-01 9.13298425e-02
 2.83553653e-01 8.32819406e-02]
[1.93799237e+01 2.16945802e+01 1.28231298e+02 1.18592977e+03
 1.01294580e-01 1.48612977e-01 1.76939466e-01 1.00698779e-01
 1.91539695e-01 6.06029008e-02 7.42803817e-01 1.22253817e+00
 5.25058015e+00 9.56781679e+01 6.59868702e-03 3.21766947e-02
 4.24197710e-02 1.56739847e-02 2.03039695e-02 3.95338931e-03
 2.37094656e+01 2.89126718e+01 1.58496183e+02 1.75302290e+03
 1.40424733e-01 3.57757710e-01 4.49306107e-01 1.92431069e-01
 3.11881679e-01 8.61654962e-02]


Take the predictions produced by K-means, and compare them with the ```labels``` variable (i.e., the ground truth from our dataset).  

Compute how many data points in the dataset have been clustered correctly (i.e., positive cases in one cluster, negative cases in the other).

*HINT*: you can use ```np.count_nonzero(series_a == series_b)``` to quickly compute the element-wise comparison of two series.

**IMPORTANT**: K-means is a clustering algorithm, so it will not output a label for each data point, but just a cluster identifier!  As such, label ```0``` does not necessarily match the cluster identifier ```0```.


In [33]:
''' 4 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE
predict_pd=predictions.toPandas()
mis_match=np.count_nonzero(labels==predict_pd['prediction'])
Prediction_accuracy=100-((mis_match/len(predict_pd['prediction']))*100)
print("Accuracy ", Prediction_accuracy)
print("Number of data points that have been clustered correctly ", len(predict_pd) - mis_match)

                                              features  prediction
0    [17.99, 10.38, 122.8, 1001.0, 0.1184, 0.2776, ...           1
1    [20.57, 17.77, 132.9, 1326.0, 0.08474, 0.07864...           1
2    [19.69, 21.25, 130.0, 1203.0, 0.1096, 0.1599, ...           1
3    [11.42, 20.38, 77.58, 386.1, 0.1425, 0.2839, 0...           0
4    [20.29, 14.34, 135.1, 1297.0, 0.1003, 0.1328, ...           1
..                                                 ...         ...
564  [21.56, 22.39, 142.0, 1479.0, 0.111, 0.1159, 0...           1
565  [20.13, 28.25, 131.2, 1261.0, 0.0978, 0.1034, ...           1
566  [16.6, 28.08, 108.3, 858.1, 0.08455, 0.1023, 0...           0
567  [20.6, 29.33, 140.1, 1265.0, 0.1178, 0.277, 0....           1
568  [7.76, 24.54, 47.92, 181.0, 0.05263, 0.04362, ...           0

[569 rows x 2 columns]
Accuracy  85.41300527240773
Number of data points that have been clustered correctly  486


Now perform dimensionality reduction on the ```features``` using the [PCA](https://spark.apache.org/docs/latest/ml-features.html#pca) statistical procedure, available as well in MLlib.

Set the ```k``` parameter to **2**, effectively reducing the dataset size of a **15X** factor.

In [34]:
''' 6 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors

pca = PCA(k=2, inputCol="features", outputCol="pcaFeatures")
pca_model = pca.fit(features)
pcaFeatures = pca_model.transform(features).select("pcaFeatures")

kmeans = KMeans(featuresCol='pcaFeatures').setK(2).setSeed(1)
pca_kmeans_model = kmeans.fit(pcaFeatures)
pca_predictions = pca_kmeans_model.transform(pcaFeatures)
pca_predictions.take(30)

[Row(pcaFeatures=DenseVector([-2260.0139, -187.9603]), prediction=1),
 Row(pcaFeatures=DenseVector([-2368.9938, 121.5874]), prediction=1),
 Row(pcaFeatures=DenseVector([-2095.6652, 145.114]), prediction=1),
 Row(pcaFeatures=DenseVector([-692.6905, 38.5769]), prediction=0),
 Row(pcaFeatures=DenseVector([-2030.2125, 295.298]), prediction=1),
 Row(pcaFeatures=DenseVector([-888.2801, 26.0798]), prediction=0),
 Row(pcaFeatures=DenseVector([-1921.0822, 58.8076]), prediction=1),
 Row(pcaFeatures=DenseVector([-1074.7813, 31.7712]), prediction=0),
 Row(pcaFeatures=DenseVector([-908.5785, 63.8308]), prediction=0),
 Row(pcaFeatures=DenseVector([-861.5784, 40.5707]), prediction=0),
 Row(pcaFeatures=DenseVector([-1404.5591, 88.2322]), prediction=0),
 Row(pcaFeatures=DenseVector([-1524.2324, -3.2631]), prediction=1),
 Row(pcaFeatures=DenseVector([-1734.3856, 273.1627]), prediction=1),
 Row(pcaFeatures=DenseVector([-1162.914, 217.6348]), prediction=0),
 Row(pcaFeatures=DenseVector([-903.4301, 135.615

Now run K-means with the same parameters as above, but on the ```pcaFeatures``` produced by the PCA reduction you just executed.

Compute the Silhouette score, as well as the number of data points that have been clustered correctly.

In [35]:
''' 11-13 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
kmeans=KMeans().setK(2).setSeed(1).setFeaturesCol("pcaFeatures")
model = kmeans.fit(pcaFeatures)
kmeans.getFeaturesCol()
predictions = model.transform(pcaFeatures)
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(pca_predictions.withColumnRenamed('pcaFeatures','features'))
print("Silhouette score = " + str(silhouette))
Prediction_accuracy=100-((mis_match/len(predict_pd['prediction']))*100)
print("Accuracy ", Prediction_accuracy)
mis_match=np.count_nonzero(labels==predict_pd['prediction'])
print("Number of data points that have been clustered correctly ", len(predict_pd) - mis_match)

Silhouette score = 0.8348610363444832
Accuracy  85.41300527240773
Number of data points that have been clustered correctly  486


Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!